In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#! pip install mosestokenizer

In [3]:
# colab　にmecabを入れる
#!apt install aptitude
#!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
#!pip install mecab-python3==0.7

In [4]:
import sys
import gc
import os
#base_dir = "drive/My Drive/jp_en_translation/".replace("/",os.sep)
base_dir = "../"
emb_dir = "drive/My Drive/util_data/".replace("/",os.sep)
sys.path.append("/Users/ueki/Desktop/work/jp_en_translation")
sys.path.append("/Users/ueki/Desktop/JP_EN_Transalation")
sys.path.append(base_dir)
sys.path.append("drive/My Drive/jp_en_translation".replace("/",os.sep))

In [5]:
from models.S2S_simple import Seq2Seq
from utils.LangEn import LangEn
from utils.LangJa import LangJa
from utils.build_emb import build_en_emb,build_jp_emb
from utils.preprocess import loadLangs
import numpy as np
from keras.utils import np_utils
import matplotlib.pyplot as plt
from keras.preprocessing import sequence
import torch

Using TensorFlow backend.


In [6]:
config={
    "corpus_file":base_dir+"data/jpn.txt",
    "en_col":"description_en",
    "jp_col":"description_jp",
    "mask_token":0,
    "SOS_token":1,
    "EOS_token":2,
    "UNK_token":3,
    "max_features":8000,
    "MAX_LENGTH":20,
    "train_size":5000,
    "val_size":300,
    "batch_size":128,
    "epochs":1,
    "maxlen_enc":20,
    "maxlen_dec":20,
    "n_hidden":500,
    "input_dim":5000,
    "jp_voc":8000,
    "en_voc":5000,
    "output_dim":8000,
    "emb_dim":300,
    "use_enc_emb":False,
    "use_dec_emb":False,
    "validation_split":0.01,
    "trained_param_dir":base_dir+"trained_models/1_lstm_ja_en_01.hdf5",
    "translate_length":25,
    "en_W2V_FILE" : emb_dir+"GoogleNews-vectors-negative300.bin.gz",
    "jp_W2V_FILE":emb_dir+"ja_data/ja.bin",
    "src":"en",
    "trg":"jp",
    "learning_rate":0.01,
}

# train

In [7]:
data=loadLangs(config)

reading lines


In [8]:
val_data = data[config["train_size"]:config["train_size"]+config["val_size"]]
data = data[:config["train_size"]]

In [9]:
EN_lang = LangEn(config)
JP_lang = LangJa(config)

stdbuf was not found; communication with perl may hang due to stdio buffering.


In [10]:
for s in data[config["en_col"]]:
    EN_lang.addSentence(s)

In [11]:
for s in data[config["jp_col"]]:
    JP_lang.addSentence(s)

## input の加工

In [12]:
if config["src"]=="jp":
    src_col=config["jp_col"]
    trg_col=config["en_col"]
    Langs={"src":JP_lang,"trg":EN_lang}
else:
    src_col=config["en_col"]
    trg_col=config["jp_col"]
    Langs={"trg":JP_lang,"src":EN_lang}

In [13]:
input_en = data[src_col]

In [14]:
input_source_lang=data[src_col].apply(lambda x:Langs["src"].word2id(x))
input_target_lang=data[trg_col].apply(lambda x:Langs["trg"].word2id(x))

In [15]:
val_source = val_data[src_col].apply(lambda x:Langs["src"].word2id(x))
val_target = val_data[trg_col].apply(lambda x:Langs["trg"].word2id(x))

In [16]:
input_source_padded=sequence.pad_sequences(input_source_lang,maxlen=config["MAX_LENGTH"],padding="post",truncating="post")
input_target_padded=sequence.pad_sequences(input_target_lang,maxlen=config["MAX_LENGTH"],padding="post",truncating="post")

In [17]:
val_source_padded=sequence.pad_sequences(val_source,maxlen=config["MAX_LENGTH"],padding="post",truncating="post")
val_target_padded=sequence.pad_sequences(val_target,maxlen=config["MAX_LENGTH"],padding="post",truncating="post")

In [18]:
input_source_padded[10]

array([51, 57, 58, 43, 59, 60, 61, 28, 62,  6,  2,  0,  0,  0,  0,  0,  0,
        0,  0,  0], dtype=int32)

In [19]:
input_target_padded[10]

array([32, 23, 63, 40, 27, 64, 65, 10, 66, 12, 67,  6,  2,  0,  0,  0,  0,
        0,  0,  0], dtype=int32)

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [30]:
w2i = EN_lang.word2index
w2c = EN_lang.word2count

In [31]:
top_freq = sorted(w2c.items() ,key = lambda x:x[1],reverse=True)

In [33]:
len(top_freq)

3663

In [34]:
EN_lang.n_words

3667

In [35]:
len(w2i)

3667

## create embedding

In [21]:
#en_emb = build_en_emb(config,EN_lang.word2index)

In [22]:
#ja_emb = build_jp_emb(config,JP_lang.word2index)

In [23]:
#if config["src"]=="en":
#    emb={"src_emb":en_emb,"trg_emb":ja_emb}
#else:
  #  emb={"src_emb":ja_emb,"trg_emb":en_emb}

In [24]:
#print("enc->",emb["trg_emb"].shape)
#print("dec->",emb["src_emb"].shape)

In [25]:
#trainer = Seq2Seq(config,enc_emb=emb["src_emb"],dec_emb=emb["trg_emb"])
trainer = Seq2Seq(config)

In [26]:
#trainer.trainIters(input_source_padded,input_target_padded,val_source_padded,val_target_padded)

# test

In [27]:
val_padded=sequence.pad_sequences(val_data_id,maxlen=config["MAX_LENGTH"],padding="post",truncating="post")

NameError: name 'val_data_id' is not defined

In [ ]:
ret=trainer.translateIter(val_source_padded[:10])

In [ ]:
ret[0]

In [ ]:
Langs["trg"].id2word(ret[0])

In [ ]:
ret=[]
for s in val_source_padded[:10]:
    ret.append(trainer.translateIter(s))

In [ ]:
ret

In [ ]:
for src,pred,target in zip(val_data[src_col],ret,val_data[trg_col]):
    print("src->",src)
    print()
    print("pred->"," ".join(Langs["trg"].id2word(pred[0])))
    print("ans->",target)
    print("------------------")